In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
import pandas as pd

# Due to the sensitive data, I can't share it in the repo. Feel free to use your own or open source data.

syn_df = pd.read_csv("/content/synthetic_data.csv")

In [ ]:
syn_df.shape

In [ ]:
syn_df_dict = syn_df.to_dict(orient="records")

In [ ]:
syn_df_dict[0]

In [ ]:
syn_dataset = []

for data in syn_df_dict:
    topic = data['Topic'].title()
    description = data['Desciption']

    Fully_Compliant = f"You are writing a historical project submission that is fully compliant with the RFI topic: {topic}.\n\
        Include all the requirements as specified in the RFI description below. Assume that all practices were followed exactly as requested.\n\n\
        RFI Description:\n{description}\n\n\
        Resposne:\n"

    Partially_Compliant = f"You are writing a historical project submission that is only partially compliant with the RFI topic: {topic}.\n\
        Include some of the RFI requirements, but omit or deviate from others. Be realistic and explain why certain practices may not have been followed fully (e.g., constraints, legacy standards, partial implementation).\n\n\
        RFI Description:\n{description}\n\n\
        Response:\n"

    Non_Compliant = f"You are writing a historical project submission that is non-compliant with the RFI topic: {topic}.\n\
        The response should fail to meet most of the specified RFI requirements. Include vague statements, incorrect practices, or outdated methods. Do not mention full compliance or best practices.\n\n\
        RFI Description:\n{description}\n\n\
        Response:\n"

    Enhanced_Compliant = f"You are writing a historical project submission that goes beyond full compliance for the RFI topic: {topic}.\n\
        Include all RFI requirements and add additional practices, optimizations, safety enhancements, or innovative approaches not required but beneficial.\n\n\
        RFI Description:\n{description}\n\n\
        Response:\n"

    prompts = [Fully_Compliant, Partially_Compliant, Non_Compliant, Enhanced_Compliant]
    syn_dataset.extend(prompts)

In [ ]:
len(syn_dataset)

In [ ]:
from pprint import pprint

pprint(syn_dataset[0])

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit", # Choose ANY! eg teknium/OpenHermes-2.5-Mistral-7B
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
syn_dataset[:1]

In [ ]:
from tqdm import tqdm

synthetic_data_output = []

for prompt in syn_dataset:
    outputs = model.generate(
        tokenizer([prompt], return_tensors="pt").input_ids.cuda(),
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    synthetic_data_output.append(output_text)
    entry['output'] = output_text.split("### Response:")[-1].strip()  # Customize if needed


In [ ]:
synthetic_data_output[1]

In [ ]:
syn_dataset[0]

In [ ]:
outputs = model.generate(
        tokenizer([syn_dataset[1]], return_tensors="pt").input_ids.cuda(),
        max_new_tokens=300,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
    )

    synthetic_data_output.append(output_text)
    entry['output'] = output_text.split("### Response:")[-1].strip()  # Customize if needed


In [ ]:
output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [ ]:
output_text.split("Partially Compliant resposne:")[-1].strip()

In [ ]:
output_text.split("Title:")[-1].strip()

In [ ]:
syn_dataset[0]

# End